# Stock market Indicator Using Klangoo

The stock prices are directly related to people opinion and trust. Financial markets are based on `expectations` , and investors buy or sell shares depending on the `sentiments of fear or desire` caused by some event . Investors bet on the credibility of the media outlets that publish rumours despite knowing that the information is not always reliable. Sentiment Analysis can be used to determine the overall opinion for a certain stock. 

 A fast answer can be to apply `sentiment analysis` on every text contaning the word(s) we are interrested in.

But, It is not accurate to study every text containing the keyword we are looking for, si how de we know if the domain of the text is  the same domain we we are interrested in (Buisness, Technology, ...) or the text is about a completely different domain and citing this word as an example while speaking about another thing??  

 How do we know if a certain text is directly related to the stock we are interrested in (i.e.: the stock is the main subject or one of the main subjects of the text) and not cited as a secondary subject in the text?

### KLANGOO can be the answer

 Klangoo can be used to determine, for each text (twitter tweet or a newspaper article) if the domain and the main subject of the text (entities) are what we are interrested in. 


 In the first part, we used a dataset already containng the sentiment analysis results of tweets. We used a stock market tweets from kaggle, (https://www.kaggle.com/datasets/sohelranaccselab/stock-market-tweets-data-sentiment-analysis ).
 Each row of the csv file contains a tweet details, the tweet initial text and a sentiment analysis result (positive, negative, neutral) .
 After sending the tweet as a requeest to Klangoo, we process the results: if the main category and the main entities of the tweet are appropriate: the tweet will be processed. Otherwise it will be neglected.
 A final sentiment analysis will be given to the wanted entity. If it is negative, this could be an indicator that the  stock is going down (maybe, nothing is 100% sure in stock market), and vice versa.

 The result will be a simple advice of selling, or keeping / buying a certain stock according to the general public opinion. Also a final grade over 5 will be given with the number of tweets analysed to give a certain inficator to the user.

 In the second part (real use case) an API was used to collect articles from different sources. The api results of giving atricles of a main subject and of a certain language. The results are sent one by one to Klangoo to speicify, as in first part, the main categories and entities.
 
 #### N.B. This notebook contains three consecutive improvments of the idea, the most important one is the last one. If you do not have enough time to check all of them, please jump to the last one entitled Improved Version.


### Define a function that will be used in the code

In [20]:
def have_common(x,y):
   common = False
   for value in x:
      if value in y:
         common= True
   return common

### Importing Libraries and reading the csv file a Pandas dataFrame

In [2]:
pip install klangooclient

Note: you may need to restart the kernel to use updated packages.


In [21]:
from klangooclient.MagnetAPIClient import MagnetAPIClient
import pandas as pd
import json
import string

df = pd.read_csv("tweets_labelled.csv") #Read  csv file containig 5000 tweets and the sentiment of each tweet
                                     # The csv file from kaggle contains already the sentiment analysis 
df = df[:30] #only 30 rows were processed 
print()

### Prepare connection strings and secret key

In [22]:
#prepare the strings to connect to Klangoo
ENDPOINT ='https://nlp.klangoo.com/Service.svc'
CALK = '155b99fc-758c-40c8-a8ef-5d4eb9fba616'
SECRET_KEY = 'wTMeUVHL38vGrdDBbSdSFBQOWFchPTaUnIFZBJS4'

client = MagnetAPIClient(ENDPOINT, CALK, SECRET_KEY)

### Specify wanted entities and categories to search for

In [23]:
#specify the entities and the categories we are interrsted in
wanted_stock = "Amazon stock AMZN"
wanted_entities = ['amazon', 'amzn'] #Stock(s) interrested in, This option is subjective to the user.
wanted_categories = [ 'science','general','business', 'politics','technology'] #Only these categoreis will be taken into considertion

### Code

In [160]:
n = len(df)
ctr = 0
grade_sum = 0
final_grade = 0

for i in range(n): #read all the tweets from  the dataFrame
    text = df.iloc[i,2] #get text from dataFrame
    text = text
    sentiment = df.iloc[i,3]   #get sentiment analysis result from dataFrame
        
    request =  { 'text' : text, 'lang' : 'en', 'format' : 'json' }
    
    rsp = client.callwebmethod('ProcessDocument', request, 'POST') #Use Klangoo to get all entities and categories of tweet
    info = json.loads(rsp) #transform to Dictionary
    info.fromkeys
    print("request nb:", i , "done")
    print("Text for request nb:", i, "is: ", text)
    print("Sentiment for request nb:", i, "is: ", sentiment)

    entities_nb = len(info['document']['entities'])
    categories_nb = len(info['document']['categories']) 
    
    text_entities = []   # get the key topics from text and append to this list
    text_categories = []   # get the categories from text and append to this list 
    
    for j in range(entities_nb):
        text_entities.append(info['document']['entities'][j]['token'].lower())#adding categoreis from klangoo response to  text_categories
    
    for j in range(categories_nb):
        text_categories.append(info['document']['categories'][j]['name'].lower())#adding categoreis from klangoo response to  text_categories
    
    print("Entities for request nb:", i, "are: ", text_entities)
    print("Categories for request nb:", i, "are: ", text_categories, "\n")

    check_category = have_common(wanted_categories, text_categories ) #check existecne of an item of text_categories in wanted_category 
    check_entities = have_common(wanted_entities, text_entities )
    
    if check_category  and check_entities: # if the category and topics of the text are in what we are interrested in 
        if sentiment == 'positive':
            grade_sum += 1   
        elif sentiment == 'negative':
            grade_sum -= 1
        #else is: stays zero 
        ctr += 1
           
  

if ctr != 0:
    final_grade = grade_sum / ctr

    if final_grade <0:
        print("The " , wanted_stock  ,"have negative Semantic Analysis in the given tweets, it could be an indicator \
        that this stock does not have community trust in the dtermined time.\n  After \
        processing ", ctr , " tweets, the term Netfilx or NFLX got a scrore of : " , final_grade, " on 1 .")
    elif final_grade >0:
        print("The ",wanted_stock, "  have a positive Semantic Analysis in the given tweets, it could be an indicator \
        that this stock have community trust in the dtermined time.\n  After \
        processing ", ctr , " tweets, the term Netfilx or NFLX got a scrore of : " , final_grade, " on 1 .")
    else:
        print("After analysing the specifeid texts (tweets), The ", wanted_stock, " have a neutral Semantic Analysis. ")  

request nb: 0 done
Text for request nb: 0 is:  RT @vincent13031925: Biggest mistakes of the century: From before  Apple AAPL is  only a PC maker  Amazon AMZN is only a bookstore Goo…
Sentiment for request nb: 0 is:  negative
Entities for request nb: 0 are:  ['amazon', 'apple', 'rt']
Categories for request nb: 0 are:  ['technology', 'business'] 

request nb: 1 done
Text for request nb: 1 is:  RT @BarrySchwartzBW: Carl Icahn once owned 10% of Netflix. That stake today would have been worth about 24B. He also had 52m shares of App…
Sentiment for request nb: 1 is:  neutral
Entities for request nb: 1 are:  ['carl icahn', 'netflix']
Categories for request nb: 1 are:  ['business', 'technology'] 

request nb: 2 done
Text for request nb: 2 is:  RT @gzbenso: Netflix leads its rivals in original TV shows by a wide margin in both quantity and quality  according to new data analysis ht…
Sentiment for request nb: 2 is:  positive
Entities for request nb: 2 are:  ['netflix']
Categories for request nb:

## Second Part: Real Use case (Bitcoin)

### Now, after we tried our idea on a dataset, let's try our idea on a real use case. We will get news articles from different newspapers and apply the sentiment analysis and use klangoo to make a final indicator only for the stocks we are interrested in.

### The newsapi is used here to provide articles from different sources (Better than searching in hundreds of websites) containig a specific word and in a certain date range (from -to), in ceratin websites we specify or over 80,000 websites.

### But it is not logical to take all articles containng this word into consideration. So we will use Klangoo to know the main entities and the domain of the article, then we can apply sentiment analysis.


## Bitcoin stock Indicator between 01/07/2022 and 13/07/2022

In [24]:
import requests

#get all the articles from .. to .. containin the words realted to bicoin
url = ('https://newsapi.org/v2/everything?' 
       'q=Bitcoin USD& Or q=BTC-USD& or q=bitcoin& or q=Bitcoin& or q=BTC& or q=btc&'#serach for articles containng bitcoin related words
       'from=2022-07-01&'     #search article in the last week 
       'to=2022-07-13&'
       'language=en&'
       'sortBy=popularity&'
       'apiKey=42f6576050c04962b5556e40825857df')

response = requests.get(url)
print("Request successful")

Request successful


### Install newsdata to get articles

In [333]:
pip install newsdataapi

Note: you may need to restart the kernel to use updated packages.


### Make a request to get atricles containing bitcoin related articles

In [25]:
from newsdataapi import NewsDataApiClient

api = NewsDataApiClient(apikey='pub_9173b9c5020815c947c4ebd4e19094a1b11d')
#https://newsdata.io/

response = api.news_api( q= "bitcoin OR Bitcoin OR bitcoins OR Bitcoins", language='en')
print("Done")

Done


In [26]:
print(response['results'][0])

{'title': 'Everything You Need to Know About Crypto and Tax Returns in 2022', 'link': 'https://www.lifehacker.com.au/2022/07/how-investing-in-crypto-impacts-your-tax-return/', 'keywords': ['Life', 'Money', 'Organise', 'au', 'crypto', 'finance', 'tax', 'tax time'], 'creator': ['Stephanie Nuzzo'], 'video_url': None, 'description': "Two experts share their advice on how to approach tax time if you've bought or sold crypto this financial year. Here's what you need to know. The post Everything You Need to Know About Crypto and Tax Returns in 2022 appeared first on Lifehacker Australia.", 'content': 'So, you invested in some crypto this last financial year . That’s all well and good (I mean, depending on how those investments went). But what do you do come tax time when it comes to crypto? That’s the question a whole lot of Aussies have been asking over the past couple of years, and seeing as we have tax tips on the brain right now , we thought we’d seek out some tax advice for all you crypt

### Print the frist text

In [38]:
print("First text:" , response['results'][0]['creator']) #see first text of the response

First text: ['Stephanie Nuzzo']


In [44]:
#determine the wanted entity and categories
wanted_stock_2 = "Bitcoin stock BTC" #Only for the final text print

# The top influencers in stock market prediction
Top_influencers = ['elon musk', 'nick black', 'allan mantaring', 'jordan belfort', 'tangie seals', \
                   'rowan mcinnesmark minervini', 'vic scherersteve sturner', 'brittney castro', 'garrett ashe', \
                   'raja al mazrouei', 'chamath palihapitiya', 'cathie wood', 'dave portnoy', 'oliver l. velez', \
                   'john bollinger', 'robert ross', 'kayla kilbride', 'errol coleman', 'timothy sykes', 'kritika yadav',\
                   'trajan king', 'denis stelmak', 'patrick wieland']

#The most trusted newspapers in the domain of stock exchange
Trusted_resources =['moneycontrol', 'the economic times', 'bloomberg quint', 'reuterslivemint', '\
buisness standard', 'google finance', 'ndtv profit', 'yahoo finance']
wanted_entities_2 = ['bitcoin', 'btc'] #Stock(s) interrested in, This option is subjective to the user.
wanted_categories_2 = ['general','business', 'politics','technology']#Only these categoreis will be taken into considertion
wanted_topics_2 = ['bitcoin', 'btc']

### Code

In [45]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

nb = len(response['results'])
temp_nb = 2 #For testing only
ctr_2 = 0
grade_sum_2 = 0
final_grade_2 = 0

for i in range(temp_nb ):
    #Since the API give only 200 chars in contnet, this is not acceptable
    # a good idea would be to use the title and the description as a subsitute since they contion most of the meaning.
    
    text_2 = response['results'][i]['content']
    creator = response['results'][i]['creator'] #get the creator of text from the API response
    
    if text_2 is not None:
        request_2 =  { 'text' : text_2, 'lang' : 'en', 'format' : 'json' }
        rsp_2 = client.callwebmethod('ProcessDocument', request_2, 'POST')#Use Klangoo to get all entities and categories of text
        info_2 = json.loads(rsp_2) #transform from Bytes to Json format
        info_2.fromkeys

        print("Request nb:",i,"is successful.")
        print("Text nb:", i, "is: ", text_2[:200])#print first 200 chars of the text

        entities_nb_2 = len(info_2['document']['entities'])
        keyTopics_nb_2 = len(info_2['document']['keyTopics'])
        categories_nb_2 = len(info_2['document']['categories']) 
        
        text_entities_2 = []   # get the key topics from text and append to this list
        key_topics_2 = []
        text_categories_2 = []   # get the categories from text and append to this list 
       
        
        for j in range(entities_nb_2):#adding entites from klangoo response to  text_entities_2 list
            if info_2['document']['entities'][j]['score'] > 0.33: #only add the entities with high score (not all of them), if high score the entity is directly related to main subject
                text_entities_2.append(info_2['document']['entities'][j]['token'].lower() ) 
                
        for j in range(keyTopics_nb_2):#adding keyTopics from Klangoo to keyTopics_2
            if info_2['document']['keyTopics'][j]['score'] > 0.8: #key topics contains many topics, but the high rated are directly related
                key_topics_2.append( info_2['document']['keyTopics'][j]['text'].lower())
                
        for j in range(categories_nb_2):#adding categoreis from klangoo response to  text_categories_2
            text_categories_2.append(info_2['document']['categories'][j]['name'].lower() ) # append the key categories to the list


            
        print("Entities for Text nb:", i, "are: ", text_entities_2)
        print("keyTopics for text nb: ", i, "are", key_topics_2,"\n")
        print("Categories for Text nb:", i, "are: ", text_categories_2, "\n")
        
        #check existecne of an item of text_categories in wanted_category 
        check_category_2 = have_common(wanted_categories_2, text_categories_2 ) 
        check_topics_2 = have_common(wanted_topics_2, key_topics_2)
        check_entities_2 = have_common(wanted_entities_2, text_entities_2 )
                
        #If the article category and entity are the wanted ones, we apply S.A.
        if check_category_2  and (check_entities_2 or check_topics_2): 
            ctr_2 += 1
            score_2 = analyser.polarity_scores(text_2)
            print("Sentiment analysis for text: ",  i , "is: ", score_2)
            #check if the text is from a top influencer or a top trusted source, if so consider as 10 texts
            if (creator in Top_influencers) or (creator in  Trusted_resources):
                ctr_2 += 10
                grade_sum_2 += score_2['compound'] * 10
            #compound(result from NLTK library) element of the polarity score determine if positive >0.5 or negative <-0.5
            grade_sum_2 += score_2['compound'] 
        print("check_category condition: ", check_category_2 )
        print("check_entities condition: ", check_entities_2)
        print()
        print("-----------------")
        print()
        

print("counter", ctr_2)        
if ctr_2 > 0:
    final_grade_2 = grade_sum_2 / ctr_2

    if final_grade_2 < -0.1: #check the condition neg or pos and total score in NLTK
        print("The " , wanted_stock_2  ,"have negative Semantic Analysis in the given tweets,it could be an indicator \
        that this stock does not have community trust in the dtermined time.\
        After processing ", ctr_2 , " articles, the term bitcoin or BTC got a polarity scrore of : " , final_grade_2, " on 1 .")
    elif final_grade > 0.1:
        print("The ",wanted_stock_2, "  have a positive Semantic Analysis in the given tweets,it could be an indicator \
        that this stock have community trust in the determined time.\
         After processing ", ctr_2 , " articles,the term bitcoin or BTC got a polarity scrore of :" , final_grade_2, " on 1 .")
    else:
        print("After analysing the relative articles, the ", wanted_stock_2, " have a neutral Semantic Analysis.")          
    

    


Request nb: 0 is successful.
Text nb: 0 is:  So, you invested in some crypto this last financial year . That’s all well and good (I mean, depending on how those investments went). But what do you do come tax time when it comes to crypto? That’s 
Entities for Text nb: 0 are:  ['mena theodorou', 'simon ho']
keyTopics for text nb:  0 are ['tax advice', 'tax time', 'tax rules', 'mena theodorou', 'tax', 'tax data', 'tax season', 'tax advisor', 'tax tips', 'tax pro', 'tax implications', 'one cryptocurrency'] 

Categories for Text nb: 0 are:  ['business'] 

check_category condition:  True
check_entities condition:  False

-----------------

counter 0


In [36]:
text = "ax: When it comes to NFTs and gaming tokens, Ho explained that there have been “intriguing tax issues”. “The ATO has ruled in a private ruling that NFTs are not personal use assets. However, the private ruling is based on a specific set of circumstances. Not all NFTs and gaming tokens are created equally so there is no one-size-fits-all answer, so again it’s probably best [that] the crypto/NFT holder engages with a tax advisor to know how the rules apply to their circumstances/activities,” he shared. You can see more about the ATO’s approach to NFTs here . One of the more common misconceptions Theodorou pointed to during our exchange was that some folks believe that the act of swapping one cryptocurrency for another is not taxable. “The answer couldn’t be further from the truth,” he said. “Tax is still payable on the trade even if you have not realised any of the gains in fiat . Therefore it is essential that you record the fiat value of the crypto at the time you swap it for the other crypto.” If you’re still learning how to properly approach your cryptocurrency come tax time, it’s worth having a read through the resources on the ATO website – in addition to speaking with your accountant. In fact, Assistant Commissioner Tim Loh of the ATO recently made a statement sharing, “We know crypto assets and their tax implications can seem complicated. That’s why our focus is on helping people get it right.” “Over one million taxpayers will have a message appear as a reminder when they prepare their tax returns saying they may have capital gains or capital losses from crypto to declare,” said Mr Loh. Theodorou also stated that you can explore the ATO’s private ruling feature option. According to the ATO website, this is “binding advice that sets out how a tax law applies to you in relation to a specific scheme or circumstance.” “We know of users who have had very positive experiences getting certainty from the ATO on how the tax rules apply to their crypto activities via the private ruling feature,” Theodorou shared. He explained that the “feature is often used for issues for which the answer is highly dependant on the facts e.g. whether you’re a trader, investor, or hobbyist. It’s a little-known feature that Aussies can take advantage of.” We reached out to the ATO for further guidance in this space back in 2021, and a spokesperson for the body shared the below statement: “Our main focus is on helping people to get it right this tax time. In many ways, income from cryptocurrency is no different to income from other sources. It’s important for people to know, however, that cryptocurrency transactions are not anonymous. The ATO uses the data that it receives from Designated Service Providers (DSPs) to match the data to an individual to ensure people trading in cryptocurrency are paying the right amount of tax.” So be thorough, and ask questions where necessary. Hopefully, you’re feeling a little more confident about approaching your tax return this year. When in doubt, however, just reach out to a tax pro. Lifehacker has updated this piece since its original publish date to reflect details of tax time 2022."
print(text[:200])

ax: When it comes to NFTs and gaming tokens, Ho explained that there have been “intriguing tax issues”. “The ATO has ruled in a private ruling that NFTs are not personal use assets. However, the priva


### The result of the Bitcoin sentiment analysis  in the past week (01/07/2022 -> 13/07/2022) gives us a negative indicator about "bitcon-USD" stock, and in fact the stock is having great losses in this time. It is not a scientific way to prove the validity of the indicator, but it gives a simple insight of it. 

### Improved version

Accroding to this article https://link.springer.com/article/10.1007/s12559-021-09819-8 in Springer the most accurate lexicon library for analyzing tweets for stock market prediction is senticnet, a library made by the MIT Media Laboratory in 2010.So We used the senticnet library in this version.

Added to this:  influential  accounts in the financial field  like Mark Zuckerberg, Elon musk,... will give a greater impact to people than a normal one. added to that a professional newspaper like BBC or   will have a greater impact, so this is also taken into consideration. A greater coeeficant shoud be given to these. But how? We used KLANGOO to determine the existence of these influencers or these important newspapers. 